In [1]:
import requests
from datetime import datetime
import pandas as pd


def get_github_pat():
    with open("pat.txt", "r") as file:
        pat = file.read().strip()
    return pat


access_token = get_github_pat()


def github_graphql(query):
    url = "https://api.github.com/graphql"
    json = {"query": query}
    headers = {"Authorization": "token %s" % access_token}

    r = requests.post(url=url, json=json, headers=headers)
    r.raise_for_status()
    return r.json()


def github_rest_get(url):
    headers = {
        "Authorization": f"Bearer {access_token}",
        "X-GitHub-Api-Version": "2022-11-28"
        #  "Accept": "application/vnd.github.v3+json"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

In [2]:
github_dict = {
    "LukvonStrom": "Lukas",
    "Sree5835": "Sree",
    "HanchengZuo": "Hancheng",
    "xiatianrui1110": "Tianrui",
    "Cenxn": "Mingzirui",
    "sudoPom": "Ponmile",
    "wqt123": "Qitian",
    "jasonho2582001": "Jason",
    "james-parky": "James",
}

In [3]:
org_name = "UCL-Drone-Deliveries"

project_list_query = """query{
    organization(login: """+'"'+org_name+'"'+""") {
      projectsV2(first: 20) {
        nodes {
          id
          title
          createdAt
          updatedAt
          closedAt
        }
      }
    }
  }"""
all_projects = github_graphql(project_list_query)

project_id = all_projects["data"]["organization"]["projectsV2"]["nodes"][1]["id"]

print(project_id)



project_query = '''{
  node(id: "'''+project_id+'''") {
    ... on ProjectV2 {
      items(first:100) {
        nodes {
          id
          fieldValues(first: 10) {
            nodes {
              ... on ProjectV2ItemFieldLabelValue {
                labels(first: 10){
                  nodes {
                    name
                  }
                }
                field {
                  ... on ProjectV2FieldCommon {
                    name
                  }
                }
              }
              ... on ProjectV2ItemFieldTextValue {
                text
                field {
                  ... on ProjectV2FieldCommon {
                    name
                  }
                }
              }
              ... on ProjectV2ItemFieldSingleSelectValue {
                name
                field {
                  ... on ProjectV2FieldCommon {
                    name
                  }
                }
              }
            }
          }
          content {
            ... on DraftIssue {
              title
              body
              assignees(first: 10) {
                nodes {
                  login
                }
              }
              createdAt
              updatedAt
              bodyText
            }
            ... on Issue {
              title
              assignees(first: 10) {
                nodes {
                  login
                }
              }
              createdAt
              updatedAt
              closedAt
              repository {
                name
              }
              bodyText
            }
            ... on PullRequest {
              url
              title
              author {
                login
              }
              createdAt
              updatedAt
              closedAt
              headRefName
              headRepository {
                name
              }
              bodyText
            }
          }
        }
      }
    }
  }
}'''

response = github_graphql(project_query)

print(response)

PVT_kwDOCQB6_84AYRas
{'data': {'node': {'items': {'nodes': [{'id': 'PVTI_lADOCQB6_84AYRaszgMNgPQ', 'fieldValues': {'nodes': [{}, {}, {'text': 'Modify the deployment view of crane spotting system', 'field': {'name': 'Title'}}, {'name': 'Done', 'field': {'name': 'Status'}}]}, 'content': {'title': 'Modify the deployment view of crane spotting system', 'assignees': {'nodes': [{'login': 'wqt123'}]}, 'createdAt': '2024-01-26T00:35:37Z', 'updatedAt': '2024-01-26T00:35:41Z', 'closedAt': None, 'repository': {'name': 'crane-spotting'}, 'bodyText': ''}}, {'id': 'PVTI_lADOCQB6_84AYRaszgMB3BQ', 'fieldValues': {'nodes': [{}, {'text': 'Dataset Visualization', 'field': {'name': 'Title'}}, {'name': 'Done', 'field': {'name': 'Status'}}]}, 'content': {'title': 'Dataset Visualization', 'body': '', 'assignees': {'nodes': [{'login': 'Cenxn'}]}, 'createdAt': '2024-01-18T21:25:37Z', 'updatedAt': '2024-01-18T21:25:49Z', 'bodyText': ''}}, {'id': 'PVTI_lADOCQB6_84AYRaszgL-urY', 'fieldValues': {'nodes': [{}, {'te

In [4]:


data = response["data"]["node"]["items"]["nodes"]

rows = []

def convert_fieldvalues_to_dict(items):
    result = {}
    for item in items:
        # Check if 'text' and 'field' keys exist
        if 'text' in item and 'field' in item and 'name' in item['field']:
            key = item['field']['name']
            value = item['text']
            result[key] = value
        # Check if 'name' and 'field' keys exist
        elif 'name' in item and 'field' in item and 'name' in item['field']:
            key = item['field']['name']
            value = item['name']
            result[key] = value
        elif 'field' in item and item['field']['name'] == 'Labels':
            key = item['field']['name']
            value = []
            for label in item['labels']['nodes']:
                value.append(label['name'])
            result[key] = value
    return result


# Extracting and processing data
for item in data:
    if item:
        type = "Task"
        item_id = item["id"]
        title = item["content"]["title"]
        repo = ""
        if "repository" in item["content"]:
            repo = item["content"]["repository"]["name"]
        if "body" in item["content"]:
            body = item["content"]["body"]
        else:
            body = ""
        status = "Not Started"

        field_values = convert_fieldvalues_to_dict(item["fieldValues"]["nodes"])

        if not title:
            if "Title" in field_values:
                title = field_values["Title"]

        if "Status" in field_values:
            status = field_values["Status"]
        if "url" in item["content"]:
            type = "Pull Request"
    
        created_at = item["content"]["createdAt"]
        updated_at = item["content"]["updatedAt"]

        if "closedAt" in item["content"] and item["content"]["closedAt"]:
            closed_at = item["content"]["closedAt"]
            closed_at_date = datetime.strptime(closed_at, "%Y-%m-%dT%H:%M:%SZ")
            updated_at_date = datetime.strptime(updated_at, "%Y-%m-%dT%H:%M:%SZ")
            if closed_at_date > updated_at_date:
                updated_at = closed_at

        # Its a PR
        if "author" in item["content"]:
            rows.append(
                {
                    "Id": item_id,
                    "Type": type,
                    "Title": title,
                    "Body": body,
                    "Assignee": github_dict.get(
                        item["content"]["author"]["login"],
                        item["content"]["author"]["login"],
                    ),
                    "Status": status,
                    "Date": updated_at,
                    "Branch": item["content"]["headRefName"],
                    "Repository": item["content"]["headRepository"]["name"],
                    "Labels": field_values.get("Labels", []),
                    "Description": item["content"]["bodyText"],
                }
            )
        else:
            for assignee in item["content"]["assignees"]["nodes"]:
                # Append each assignee as a separate row
                rows.append(
                    {
                        "Id": item_id,
                        "Type": type,
                        "Title": title,
                        "Body": body,
                        "Assignee": github_dict.get(
                            assignee["login"], assignee["login"]
                        ),
                        "Status": status,
                        "Date": updated_at,
                        "Repository": repo,
                        "Labels": field_values.get("Labels", []),
                        "Description": item["content"]["bodyText"],
                    }
                )


In [5]:
# Get all Repositories, then create a commit entry for each commit


getAllRepos = github_graphql('''query {
  organization(login: "'''+org_name+'''") {
    repositories(first: 100) {
      edges {
        node {
          name
          description
          url
        }
      }
      pageInfo {
        endCursor
        hasNextPage
      }
    }
  }
}
''')

def get_branches(repo_name):
  return github_graphql('''query {
  repository(name: "'''+repo_name+'''", owner: "'''+org_name+'''") {
    refs(refPrefix: "refs/heads/", first: 100) {
      edges {
        node {
          name
        }
      }
      pageInfo {
        endCursor
        hasNextPage
      }
    }
  }
}''')

def get_commits(repo_name, branch_name, since_date= "2024-01-13T00:00:00Z"):

  return github_graphql('''query {
    repository(name: "'''+repo_name+'''", owner: "'''+org_name+'''") {
      ref(qualifiedName: "'''+branch_name+'''") {
        target {
          ... on Commit {
            history(since: "'''+since_date+'''") {
              edges {
                node {
                  message
                  author {
                    user {
                      login
                    }
                  }
                  committedDate
                  authoredDate
                }
              }
            }
          }
        }
      }
    }
  }
  ''')

for repo in getAllRepos["data"]["organization"]["repositories"]["edges"]:
  repo_name = repo["node"]["name"]
  branches = get_branches(repo_name)
  for branch in branches["data"]["repository"]["refs"]["edges"]:
    branch_name = branch["node"]["name"]
    commits = get_commits(repo_name, branch_name)

    for commit in commits["data"]["repository"]["ref"]["target"]["history"]["edges"]:
      if commit["node"]["author"] is None or not "user" in commit["node"]["author"] or commit["node"]["author"]["user"] is None:
        continue
      else:
        authored_date_str = commit["node"]["authoredDate"]
        commited_date_str = commit["node"]["committedDate"]
        authored_date = datetime.strptime(authored_date_str, "%Y-%m-%dT%H:%M:%SZ")
        commited_date = datetime.strptime(commited_date_str, "%Y-%m-%dT%H:%M:%SZ")
        # Use the later date, compare them by serializing them as they come in strings like this: 2024-01-14T14:03:52Z
        if commited_date > authored_date:
          authored_date_str = commited_date_str

        rows.append({
          "Type": "Commit",
          "Title": commit["node"]["message"],
          "Body": "",
          "Assignee": github_dict.get(commit["node"]["author"]["user"]["login"], commit["node"]["author"]["user"]["login"]),
          "Status": "Done",
          "Branch": branch_name,
          "Repository": repo_name,
          "Date": authored_date_str
        })

In [6]:

# Creating a DataFrame
df = pd.DataFrame(rows)

# Assuming 'df' is your original DataFrame
# Convert 'Date' column to datetime without timezone
df['Date'] = pd.to_datetime(df['Date'], format="ISO8601", errors='raise')

# Filter out 'actions-user'
df_filtered_since_friday_no_techusers = df[df["Assignee"] != "actions-user"].copy()

# Set 'Date' as the index
df_filtered_since_friday_no_techusers.set_index('Date', inplace=True)

# Ensure the index is timezone-naive
df_filtered_since_friday_no_techusers.index = df_filtered_since_friday_no_techusers.index.tz_localize(None)

# Define 'friday' as a timezone-naive timestamp
friday = pd.Timestamp("2024-01-19").tz_localize(None)

# Filter the DataFrame to include data since Friday
df_filtered_since_friday = df_filtered_since_friday_no_techusers[df_filtered_since_friday_no_techusers.index >= friday].copy().reset_index(drop=True)

df_filtered_since_friday.reset_index(drop=True, inplace=True)



In [7]:
assignee_order = ['Ponmile', 'Lukas', 'Jason', 'James', 'Sree', 'Qitian', 'Mingzirui', 'Tianrui', 'Hancheng']

# Convert 'Assignee' to a categorical type with the specified order
df_filtered_since_friday['Assignee'] = pd.Categorical(df_filtered_since_friday['Assignee'], categories=assignee_order, ordered=True)

# Sort the DataFrame by 'Assignee'
df_filtered_since_friday = df_filtered_since_friday.sort_values('Assignee')

In [29]:
all_text =  ""

# Preprocessing steps
df_filtered_since_friday["Title"] = df_filtered_since_friday["Title"].fillna("").astype(str)
df_filtered_since_friday["Repository"] = df_filtered_since_friday["Repository"].fillna("").astype(str)
df_filtered_since_friday["Description"] = df_filtered_since_friday["Description"].fillna("").astype(str)

df_final = df_filtered_since_friday.loc[df_filtered_since_friday['Status'] != "Todo"].copy()
df_final.reset_index(drop=True, inplace=True)

for selected_teammember in df_final["Assignee"].dropna().unique():
    # Filtering the DataFrame for the current assignee
    df_filtered = df_final[df_final["Assignee"] == selected_teammember]
    non_commit_df = df_filtered[df_filtered["Type"] != "Commit"].copy()

    # Counting occurrences
    pr_count = (df_filtered["Type"] == "Pull Request").sum()
    task_count = (df_filtered["Type"] == "Task").sum()
    commit_count = (df_filtered["Type"] == "Commit").sum()

    # topics
    topics_df = df_filtered[df_filtered["Type"] != "Commit"][["Title", "Status"]]
    topics_df_sorted = topics_df.sort_values(by="Status")
    topics = "\n    - ".join(
        [f'"{title}" ({status})' for title, status in topics_df_sorted.values]
    )

    empty_description_mask = non_commit_df["Description"] == ""
    items_without_acceptance_criteria = non_commit_df[empty_description_mask]
    items_without_acceptance_criteria = len(items_without_acceptance_criteria)
    # Repos
    repos = df_filtered.loc[df_filtered["Type"] == "Commit", "Repository"].dropna().unique()
    repos_list = [repo for repo in repos if repo]
    repos_str = ", ".join(repos_list) if repos_list else "No specific repositories"

    text = f"""{selected_teammember}, these are your weekly GitHub stats:
{pr_count} Pull Requests
{task_count} Tasks
{commit_count} Commits ({repos_str})

	
You had {items_without_acceptance_criteria} open tasks without acceptance criteria {"- Great job!" if items_without_acceptance_criteria == 0 else "- please add acceptance criteria to all your tasks."}

Your tasks centered around the following topics:
    - {topics}

What you did well this week:

What you could do better in the future:

Next week you should focus on:

"""
    print(text)

    all_text += "\n"+text


with open("github_stats.txt", "w") as file:
    file.write(all_text)

Ponmile, these are your weekly GitHub stats:
0 Pull Requests
1 Tasks
1 Commits (crane-spotting-image-processor)

	
You had 1 open tasks without acceptance criteria - please add acceptance criteria to all your items.

Your tasks centered around the following topics:
    - "Produce alternative solution for current chosen camera" (Done)

What you did well this week:

What you could do better in the future:

Next week you should focus on:


Lukas, these are your weekly GitHub stats:
9 Pull Requests
9 Tasks
110 Commits (webapp, dropzone, crane-spotting)

	
You had 7 open tasks without acceptance criteria - please add acceptance criteria to all your items.

Your tasks centered around the following topics:
    - "Fix compiler error" (BLOCKED)
    - "Port to Cloud Functions v2 " (Done)
    - "Move mapbox-gl secret into GCP secret and only inject in buildtime" (Done)
    - "Increase test coverage " (Done)
    - "Remove Leaflet components entirely" (Done)
    - "Fix Prod Degradation" (Done)
    

#
